In [1]:
# !pip install mediapipe

In [2]:
# !pip install mat4py

# Import Libraries

In [15]:
from math import cos ,sin
import mediapipe as mp
import cv2
import glob
import numpy as np
import csv
import os
from mat4py import loadmat
import scipy.io as sio
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
import pickle

# Read Data Landmarks from Images using mediapipe and Ceate a dataset

In [16]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_drawing_styles = mp.solutions.drawing_styles

In [17]:
paths,paths_mat=[],[]
# get the path/directory
folder_dir ="E:\\Meta World\\Data and AI Track\\2. Phase 2\\1. Machie Learning\\Courses\\ITI Course\\ML 1 Supervised ML\\ML Project\\AFLW2000"

# iterate over files in
# that directory
for images in glob.iglob(f'{folder_dir}/*'):

    # check if the image ends with png
    if (images.endswith(".jpg")):
#         print(images)
        paths.append(images)

In [18]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 

    for idx, file in enumerate(paths):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#         results = holistic.process(image)
#         print(results.face_landmarks)
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        break

In [19]:
num_coords = len(results.face_landmarks.landmark)
num_coords

468

In [20]:
landmarks = []
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val)]
# landmarks
angles=['pitch','yaw','roll']
data = landmarks+angles

In [21]:
with open('landmarks_data.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(data)

In [22]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for idx, file in enumerate(paths):
        count=0
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
#         Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])

            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).flatten())
            
            random_file = file.split('\\')[-1].replace('.jpg' , '')
            mat_file = sio.loadmat('E:\\Meta World\\Data and AI Track\\2. Phase 2\\1. Machie Learning\\Courses\\ITI Course\\ML 1 Supervised ML\\ML Project\\AFLW2000\\'+ random_file+ '.mat')
            pose_para = mat_file["Pose_Para"][0][:3]
            pitch = pose_para[0]
            yaw = pose_para[1]
            roll = pose_para[2]
            face_row.insert(((468*2)+0),pitch)
            face_row.insert(((468*2)+1),yaw)
            face_row.insert(((468*2)+2),roll)
            
            with open('landmarks_data.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(face_row) 
                
        except:
            pass

# Read Data

In [23]:
df=pd.read_csv('landmarks_data.csv')
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y465,x466,y466,x467,y467,x468,y468,pitch,yaw,roll
0,-0.159295,0.300003,-0.169120,0.067671,-0.146352,0.109180,-0.190952,-0.301066,-0.161892,-0.022124,...,-0.640003,0.132219,-0.604989,0.751723,-0.761274,0.807323,-0.806983,-0.399231,0.018227,0.085676
1,-0.334478,0.261430,-0.472663,-0.051103,-0.284650,0.053159,-0.358159,-0.332285,-0.480441,-0.137888,...,-0.525273,0.037087,-0.505745,0.562256,-0.565438,0.632099,-0.617696,0.470065,1.189533,0.300959
2,-0.981481,1.076963,-0.956155,0.954189,-0.976846,0.985318,-1.013231,0.778098,-0.954629,0.903266,...,0.634062,-0.919076,0.647000,-0.675460,0.588512,-0.651586,0.567591,-0.184650,0.881137,-0.236852
3,0.127447,0.433119,-0.086203,0.200610,0.035008,0.228611,-0.239562,-0.172509,-0.133581,0.104239,...,-0.642603,0.021166,-0.595805,0.629668,-0.972822,0.673793,-1.040024,-0.175379,0.299208,-0.373374
4,-0.078761,0.400796,-0.344684,0.090758,-0.104594,0.147864,-0.317465,-0.235742,-0.383646,-0.000681,...,-0.595326,-0.004369,-0.552671,0.427108,-0.790101,0.482372,-0.853066,-0.882169,1.198003,-1.033374


In [24]:
X = df.iloc[:,:-3]
X

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,x464,y464,x465,y465,x466,y466,x467,y467,x468,y468
0,-0.159295,0.300003,-0.169120,0.067671,-0.146352,0.109180,-0.190952,-0.301066,-0.161892,-0.022124,...,0.229691,-0.674880,0.171099,-0.640003,0.132219,-0.604989,0.751723,-0.761274,0.807323,-0.806983
1,-0.334478,0.261430,-0.472663,-0.051103,-0.284650,0.053159,-0.358159,-0.332285,-0.480441,-0.137888,...,0.164270,-0.554017,0.101750,-0.525273,0.037087,-0.505745,0.562256,-0.565438,0.632099,-0.617696
2,-0.981481,1.076963,-0.956155,0.954189,-0.976846,0.985318,-1.013231,0.778098,-0.954629,0.903266,...,-0.887602,0.617957,-0.910065,0.634062,-0.919076,0.647000,-0.675460,0.588512,-0.651586,0.567591
3,0.127447,0.433119,-0.086203,0.200610,0.035008,0.228611,-0.239562,-0.172509,-0.133581,0.104239,...,0.111017,-0.696716,0.059032,-0.642603,0.021166,-0.595805,0.629668,-0.972822,0.673793,-1.040024
4,-0.078761,0.400796,-0.344684,0.090758,-0.104594,0.147864,-0.317465,-0.235742,-0.383646,-0.000681,...,0.106315,-0.646109,0.053495,-0.595326,-0.004369,-0.552671,0.427108,-0.790101,0.482372,-0.853066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,0.251512,0.590853,0.461883,0.140846,0.298832,0.237600,0.282476,-0.449626,0.490399,-0.017429,...,0.562673,-1.055603,0.513329,-0.992107,0.506643,-0.937817,1.226301,-1.203889,1.290611,-1.291007
1809,-0.078189,0.737183,-0.373762,0.277189,-0.126283,0.392351,-0.403332,-0.260956,-0.409893,0.125247,...,0.402032,-0.846428,0.291610,-0.780281,0.201979,-0.726654,1.212682,-1.009682,1.309340,-1.116765
1810,-0.114197,0.737743,-0.063789,0.359366,-0.069192,0.433404,-0.125912,-0.199374,-0.045589,0.222368,...,0.496784,-0.733691,0.399499,-0.683381,0.341957,-0.635232,1.295168,-0.828151,1.377284,-0.875719
1811,0.653975,1.071096,1.051907,0.496135,0.596450,0.681730,0.693948,-0.081565,1.086181,0.317043,...,0.287069,-0.829216,0.304238,-0.736888,0.380352,-0.661870,0.426355,-0.989492,0.423539,-1.098939


In [25]:
y = df.iloc[:,-3:]
y

,pitch,yaw,roll
0,-0.399231,0.018227,0.085676
1,0.470065,1.189533,0.300959
2,-0.184650,0.881137,-0.236852
3,-0.175379,0.299208,-0.373374
4,-0.882169,1.198003,-1.033374
...,...,...,...
1808,-0.367547,-0.429723,0.122791
1809,-0.156035,0.567114,-0.108536
1810,-0.197102,-0.070430,0.105118
1811,-0.232617,-1.418751,0.175960


In [26]:
y_p = y['yaw']
y_r = y['pitch']
y_y = y['roll']
y_y

0       0.085676
1       0.300959
2      -0.236852
3      -0.373374
4      -1.033374
          ...   
1808    0.122791
1809   -0.108536
1810    0.105118
1811    0.175960
1812   -0.024546
Name: roll, Length: 1813, dtype: float64

In [27]:
# # get the highly correlated features with the yaw
# c = df.corr()

# yaw_features = c['yaw'][:].abs().sort_values(ascending = False)[1:50]

# yaw_features = [*yaw_features.index]

# yaw_features_df = df[yaw_features]

# y_yaw = df['yaw']

## Modeling

In [29]:
X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(X, y_p, test_size=0.2,shuffle=True, random_state=1234)

X_val_p, X_test_p, y_val_p, y_test_p = train_test_split(X_val_p, y_val_p, test_size=0.5,shuffle=True, random_state=1234)

In [30]:
X_train_y, X_val_y, y_train_y, y_val_y = train_test_split(X, y_y, test_size=0.2, shuffle=True,random_state=1234)

X_val_y, X_test_y, y_val_y, y_test_y = train_test_split(X_val_y, y_val_y, test_size=0.5, shuffle=True,random_state=1234)

In [31]:
X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(X, y_r, test_size=0.2,shuffle=True, random_state=1234)

X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_val_r, y_val_r, test_size=0.5,shuffle=True, random_state=1234)

## used models

In [32]:
from sklearn.linear_model import Ridge

r = Ridge(alpha=0.02,random_state=42)

In [33]:
#RadiusNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor

rr = RadiusNeighborsRegressor()

In [34]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [35]:
#KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor

kr = KNeighborsRegressor()

In [36]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=.6, min_samples_leaf=5,n_estimators=100,max_depth=100, random_state=42)


In [37]:
#SVR
from sklearn.svm import SVR

svm = SVR()

In [38]:
#AdaBoostRegressor
from sklearn.ensemble import AdaBoostRegressor

ar = AdaBoostRegressor()

In [39]:
#GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()

In [40]:
#ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor()

In [41]:
#HistGradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

hr = HistGradientBoostingRegressor()

In [42]:
#BaggingRegressor
from sklearn.ensemble import BaggingRegressor

br = BaggingRegressor(base_estimator=ExtraTreesRegressor(),n_estimators=10, random_state=0)

In [43]:
#VotingRegressor
from sklearn.ensemble import VotingRegressor

# vr =  VotingRegressor(estimators=estimators)

In [44]:
# #StackingRegressor
# from sklearn.ensemble import StackingRegressor
# sr = StackingRegressor(
#     estimators=estimators,
#     final_estimator=RandomForestRegressor(n_estimators=10,
#                                           random_state=42))

# Voting Regressor

In [45]:
# ('RandomForest',rf)
# ('LinearRegression',lr)
# ('GradientBoosting',gbr)
# ('RadiusNeighbors',rr)
# ('Ridge',r)
# ('KNeighbors',kr)
# ('AdaBoost',ar)
# ('ExtraTrees',etr)
# ('HistGradientBoosting',hr)
# ('Bagging',br)

In [46]:
# choosing best models for yaw
pipelines = {
    'rf_pca':make_pipeline(MinMaxScaler(),PCA(n_components=0.98),RandomForestRegressor())
}

In [47]:
yaw_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_y, y_train_y)
    yaw_fit_models[algo] = model

In [48]:
Train_pred_yaw      = yaw_fit_models['rf_pca'].predict(X_train_y)
Validation_pred_yaw = yaw_fit_models['rf_pca'].predict(X_val_y)
yaw_hat=yaw_fit_models['rf_pca'].predict(X_test_y)

In [49]:
print("Train MSE "       , mean_squared_error( y_train_y,Train_pred_yaw ))
print("Validation MSE "  , mean_squared_error( y_val_y,Validation_pred_yaw ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_y,Train_pred_yaw))
print("Validation r2_score "       , r2_score( y_val_y, Validation_pred_yaw))
print("Test r2_score "             , r2_score(y_test_y , yaw_hat))
print("\n###########################\n")

print("Train score "                   , yaw_fit_models['rf_pca'].score(X_train_y , y_train_y))
print("Validation Score "              , yaw_fit_models['rf_pca'].score(X_val_y, y_val_y))
print("test score: "                   ,yaw_fit_models ['rf_pca'].score(X_test_y, y_test_y))
 
print("\n###########################\n")

Train MSE  0.07011767877845734
Validation MSE  0.01889479395966495

###########################

Train r2_score  0.8609840846205912
Validation r2_score  0.750181000719865
Test r2_score  0.6958898391092394

###########################

Train score  0.8609840846205912
Validation Score  0.750181000719865
test score:  0.6958898391092394

###########################



In [52]:
import pickle 

with open('yaw_model.pkl', 'wb') as f:
    pickle.dump(yaw_fit_models ['rf_pca'], f)

In [53]:
# choosing best models for roll
pipelines = {
    'rf_pca':make_pipeline(MinMaxScaler(),PCA(n_components=0.98), VotingRegressor(estimators=[('RF',rf),('Bagging',br),('Boosting',gbr),('SVM',svm)]))
}

In [54]:
roll_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_r, y_train_r)
    roll_fit_models[algo] = model

In [55]:
Train_pred_roll  = roll_fit_models['rf_pca'].predict(X_train_r)
Validation_pred_roll = roll_fit_models['rf_pca'].predict(X_val_r)
roll_hat=roll_fit_models['rf_pca'].predict(X_test_r)

In [56]:
print("Train MSE "       , mean_squared_error( y_train_r,Train_pred_roll ))
print("Validation MSE "  , mean_squared_error( y_val_r,Validation_pred_roll ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_r,Train_pred_roll))
print("Validation r2_score "       , r2_score( y_val_r, Validation_pred_roll))
print("Test r2_score "             , r2_score(y_test_r , roll_hat))
print("\n###########################\n")

print("Train score "                   , roll_fit_models['rf_pca'].score(X_train_r , y_train_r))
print("Validation Score "              , roll_fit_models['rf_pca'].score(X_val_r, y_val_r))
print("test score: "                   ,roll_fit_models ['rf_pca'].score(X_test_r, y_test_r))
 
print("\n###########################\n")

Train MSE  0.10901386362921467
Validation MSE  0.022450769702316943

###########################

Train r2_score  0.6982427967659666
Validation r2_score  0.6496881147826151
Test r2_score  0.6042669000693652

###########################

Train score  0.6982427967659666
Validation Score  0.6496881147826151
test score:  0.6042669000693652

###########################



In [57]:
import pickle 

with open('roll_model.pkl', 'wb') as f:
    pickle.dump(roll_fit_models ['rf_pca'], f)

In [58]:
# choosing best models for pitch
pipelines = {
    'rf_pca':make_pipeline(MinMaxScaler(),PCA(n_components=0.99), RandomForestRegressor())
}

In [59]:
pitch_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_p, y_train_p)
    pitch_fit_models[algo] = model

In [60]:
Train_pred_pitch      = pitch_fit_models['rf_pca'].predict(X_train_p)
Validation_pred_pitch = pitch_fit_models['rf_pca'].predict(X_val_p)
pitch_hat=pitch_fit_models['rf_pca'].predict(X_test_p)

In [61]:
print("Train MSE "       , mean_squared_error( y_train_p,Train_pred_pitch ))
print("Validation MSE "  , mean_squared_error( y_val_p,Validation_pred_pitch ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_p,Train_pred_pitch))
print("Validation r2_score "       , r2_score( y_val_p, Validation_pred_pitch))
print("Test r2_score "             , r2_score(y_test_p , pitch_hat))
print("\n###########################\n")

print("Train score "                   , pitch_fit_models['rf_pca'].score(X_train_p, y_train_p))
print("Validation Score "              , pitch_fit_models['rf_pca'].score(X_val_p, y_val_p))
print("test score: "                   ,pitch_fit_models ['rf_pca'].score(X_test_p, y_test_p))
 
print("\n###########################\n")

Train MSE  0.010368318763643776
Validation MSE  0.034584440191831184

###########################

Train r2_score  0.968952969646051
Validation r2_score  0.9006815407850709
Test r2_score  0.878298034990683

###########################

Train score  0.968952969646051
Validation Score  0.9006815407850709
test score:  0.878298034990683

###########################



In [62]:
import pickle 

with open('pitch_model.pkl', 'wb') as f:
    pickle.dump(pitch_fit_models ['rf_pca'], f)

# Demo

In [63]:
with open('yaw_model.pkl', 'rb') as f:
    yaw_model = pickle.load(f)

In [64]:
with open('roll_model.pkl', 'rb') as f:
    roll_model = pickle.load(f)

In [65]:
with open('pitch_model.pkl', 'rb') as f:
    pitch_model = pickle.load(f)

In [72]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        
        
        
        
        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).reshape(1, -1))
            
#             print(face_row)
            pitch = pitch_model.predict(face_row)
            yaw = yaw_model.predict(face_row)
            roll = roll_model.predict(face_row)
            

#             cv2_imshow(draw_axis(image,pitch,yaw,roll))
            yaw = -yaw
            tdx = NoseX
            tdy = NoseY
            size=100
    # X-Axis pointing to right. drawn in red
            x1 = size * (cos(yaw) * cos(roll)) + tdx
            y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
            x2 = size * (-cos(yaw) * sin(roll)) + tdx
            y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
            x3 = size * (sin(yaw)) + tdx
            y3 = size * (-cos(yaw) * sin(pitch)) + tdy
            cv2.line(image, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

            
        except:
            pass


        
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()